In [32]:
import numpy as np
import h5py
import pandas as pd
from tqdm import tqdm
from pyjet import cluster

from scipy.stats import binned_statistic_2d

In [2]:
filePath = "data/4m_samples.h5"
nJets = 100
nConstituents = 40

In [ ]:
def _load (filePath, nJets=200000, nConstituents=40):
    '''
    Returns:
        momenta: (nJets, 4, nConstituents)
    '''
    cols = ['E_'+str(i) for i in range(nConstituents)]+ ['PX_'+str(i) for i in range(nConstituents)] + ['PX_'+str(i) for i in range(nConstituents)] + ['PY_'+str(i) for i in range(nConstituents)] + ['PZ_'+str(i) for i in range(nConstituents)] + ['is_signal_new']

    # df = pd.read_hdf(filePath,key='data',stop=nJets, columns = cols)
    df = pd.read_hdf(filePath,key='data')
    # Take all the 4 momentum from 200 particles in all jets and reshape them into one particle per row
    momenta = df.iloc[:,:-1].to_numpy()
    momenta = momenta.reshape(-1,nConstituents,4)
    nJets = slice(nJets)
    momenta = momenta[nJets, :nConstituents, :]
    momenta = np.transpose(momenta, (0, 2, 1))
    label = df['is_signal_new']
    return momenta, label

In [3]:
cols = ['E_'+str(i) for i in range(nConstituents)]+ ['PX_'+str(i) for i in range(nConstituents)] + ['PX_'+str(i) for i in range(nConstituents)] + ['PY_'+str(i) for i in range(nConstituents)] + ['PZ_'+str(i) for i in range(nConstituents)] + ['is_signal_new']
df = pd.read_hdf(filePath,key='data',stop=nJets, columns = cols)
df

,E_0,E_1,E_2,E_3,E_4,E_5,E_6,E_7,E_8,E_9,...,PZ_31,PZ_32,PZ_33,PZ_34,PZ_35,PZ_36,PZ_37,PZ_38,PZ_39,is_signal_new
375,474.071136,103.236237,105.255569,40.176777,22.428583,20.334389,19.030899,13.460596,11.226108,10.445061,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
377,150.504532,82.257057,48.573559,47.044415,28.850746,25.893602,23.206804,19.720304,19.354494,19.159235,...,-0.598414,-1.206531,-0.381303,-0.349586,-0.295744,-0.603761,-0.154503,0.025716,0.132601,0
378,251.645386,104.147797,78.043213,60.968952,51.472054,48.337425,42.666927,41.418427,30.382032,28.893793,...,2.873340,2.019106,1.588634,1.501889,1.332423,1.290797,1.818727,0.774023,1.239125,0
379,451.566132,208.410919,190.183304,165.955811,152.131714,138.642105,115.553978,99.195404,34.859463,32.415459,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
380,399.093903,273.691956,152.837219,85.448845,50.338779,33.557629,27.275934,33.602020,23.398998,30.679037,...,-6.996005,-4.947374,-5.012867,-4.834523,-4.645840,-4.473898,-2.356860,-3.613118,-2.849185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,287.530090,168.847504,112.256172,32.228699,33.086334,33.483089,22.993778,16.389671,13.751877,12.585986,...,-2.490512,-1.689721,-1.886973,-0.757103,-2.467528,-0.969460,-0.458176,-0.826748,-1.665124,0
910,104.363907,51.634712,49.769173,45.645321,29.278572,34.209408,35.236862,30.983622,21.009840,23.172194,...,0.832889,0.554253,1.326195,1.508835,1.551696,1.375213,1.160517,1.470226,1.641878,0
914,299.944183,139.551422,29.802004,29.472048,28.298494,25.385210,21.464867,15.985082,14.983712,5.892632,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
915,872.585571,236.073547,132.817322,25.505329,23.159006,14.564881,12.370288,12.826923,10.435250,9.127288,...,-1.113016,-1.352930,-2.619766,-0.549278,-1.308018,-0.982968,-0.886416,-1.094448,-0.589929,0


In [4]:
momenta = df.iloc[:,:-1].to_numpy()
momenta = momenta.reshape(-1,nConstituents,4)
nJets = slice(nJets)
momenta = momenta[nJets, :nConstituents, :]
momenta = np.transpose(momenta, (0, 2, 1))
label = df['is_signal_new']

In [5]:
# Jet features
jetMomenta = np.sum(momenta, axis=2)
jetPt = np.linalg.norm(jetMomenta[:, 1:3], axis=1)[..., np.newaxis]
jetE = jetMomenta[:, 0][..., np.newaxis]
jetP = np.linalg.norm(jetMomenta[:, 1:], axis=1)
jetEta = 0.5 * np.log((jetP + jetMomenta[:, 3]) / (jetP - jetMomenta[:, 3]))[..., np.newaxis]
jetPhi = np.arctan2(jetMomenta[:, 2], jetMomenta[:, 1])[..., np.newaxis]
jetTheta = 2*np.arctan(np.exp(-jetEta))

# Constituent features
# delta eta, delta phi, log pT, log E,log pT / pTjet, log E / Ejet, delta R
pT = np.linalg.norm(momenta[:, 1:3, :], axis=1)
e = momenta[:, 0, :]
p = np.linalg.norm(momenta[:, 1:, :], axis=1)
eta = 0.5 * np.log((p + momenta[:, 3, :]) / (p - momenta[:, 3, :]))
etaRel = eta - jetEta
phi = np.arctan2(momenta[:, 2, :], momenta[:, 1, :])
phiRel = np.unwrap(phi - jetPhi)
dR = np.sqrt(phi ** 2 + eta ** 2)
theta = 2*np.arctan(np.exp(-eta))
cosThetaRel = np.cos(theta-jetTheta)

In [7]:
event = momenta[0,:,:]
event = np.transpose(event,(1,0))
eventCopy = np.core.records.fromarrays( [event[:,0],event[:,1],event[:,2],event[:,3]], names= 'E, PX, PY, PZ' , formats = 'f8, f8, f8,f8')

R0 = 0.2
p = 1

sequence = cluster(eventCopy, R=R0, p= p, ep=True)

In [8]:
subjet_data = event
subjet_array = sequence.inclusive_jets()


In [20]:
pT = subjet_data[:, 0]
eta = subjet_data[:, 1]
phi = subjet_data[:, 2]
mass = subjet_data[:, 3]

In [10]:
def deltaPhi(phi1,phi2):
    x = phi1-phi2
    while x >= np.pi: x -= np.pi*2.
    while x < -np.pi: x += np.pi*2.
    return x

In [25]:
# shifts all data with respect to the leading subjet so that
# the Jet Image is centerd at the origin (eta,phi) = (0,0).
eta -= subjet_array[0].eta
phi = np.array( [deltaPhi(i,subjet_array[0].phi) for i in phi])

# Rotate the jet image such that the second leading
# subjet is located at -pi/2
s1x, s1y = subjet_array[1].eta - subjet_array[0].eta, deltaPhi(subjet_array[1].phi,subjet_array[0].phi)

In [27]:
theta = np.arctan2(s1y, s1x)
if theta < 0.0:
    theta += 2 * np.pi

In [28]:
def rotate(x, y, a):
    xp = x * np.cos(a) - y * np.sin(a)
    yp = x * np.sin(a) + y * np.cos(a)
    return xp, yp

In [29]:
eta, phi = rotate(eta, phi, np.pi - theta)

In [31]:
 # Collect the trimmed subjet constituents
rjcs = np.array([pT, eta, phi, mass])
rjcs = np.expand_dims(rjcs, axis=-1)
rjcs = rjcs.transpose((1,0,2))
rjcs = np.squeeze(rjcs, axis=(2,))

In [33]:
def pixelize(jet, bins = 64, R = 1.2):
    pt, eta, phi = jet[:,0], jet[:,1], jet[:,2]

    # Define the binning for the complete calorimeter
    ranges = np.array([[-R,R],[-R,R]])

    # Sum energy deposits in each bin
    digitized = binned_statistic_2d(eta, phi, pt, statistic="sum", bins=bins, range= ranges)
#     digitized = binned_statistic_2d(eta, phi, pt, statistic="sum", bins=bins)
    
    jet_image = digitized.statistic
    return jet_image

In [ ]:
for i in rjcs:
    